In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
from scipy.stats import norm
from sklearn.mixture import GaussianMixture as GMM
from scipy import stats
import json

In [2]:
import gen_fbm_nn_model as fbm_nn

### Displacements and exponent prediction functions

In [3]:
def find_displacement(x_data, y_data, z_data, start_index=0):

    disps = np.sqrt(np.power(x_data-x_data[0],2) + np.power(y_data-y_data[0],2) + np.power(z_data-z_data[0],2))
    
    return disps 

def estimate_hurst(disps, time, window):
    
    h = np.array([])
    ht = np.array([])
    for i in range(int(window/2), len(disps)-int(1+window/2)):
        #sample of <window> points to calculate hurst exponent for
        inx = disps[(i-int(window/2)):(i+int(1+window/2))]
        #apply differencing and normalization on the data
        inx = np.array([(inx[1:]-inx[0:-1])/(np.amax(inx)-np.amin(inx))])
        test = model.predict(inx,verbose=0)
        h=np.append(h,test[0][0])
        ht = np.append(ht,time[i])
        
    return h,ht

### Downsampling functions

In [4]:
def downsample(data_input, down_int, start_index=0):
    
    data_out = data_input.iloc[start_index::down_int]
    return data_out


def dsample_est_hurst(data_in, ds_rate, window):
    """
    Args:
        data_in: pandas dataframe
        ds_rate: int, downsampling step size
    Returns:
        h_arr: 2D np array
        ht_arr: 2D np array
    """
    
    h_arr = np.empty((ds_rate,(len(data_in)//ds_rate)-(window+1)))
    ht_arr = np.empty((ds_rate,(len(data_in)//ds_rate)-(window+1)))
    
    for i in np.arange(ds_rate):
        
        downsampled_data = downsample(data_in, ds_rate, i)
        x = np.array(downsampled_data['Position X'])
        y = np.array(downsampled_data['Position Y'])
        z = np.array(downsampled_data['Position Z'])
        t = np.array(downsampled_data['Absolute Time'])
        displacements = find_displacement(x,y,z)
        h,ht = estimate_hurst(displacements, t, window)
        
        if len(h)>np.shape(h_arr)[1]:
            h=h[:-1]
            ht=ht[:-1]
            
        h_arr[i] = h
        ht_arr[i] = ht
        
    return h_arr,ht_arr


#def average_hurst(h_arr, ht_arr):
#    """
#    """
#    h_average_arr = np.mean(h_arr, axis=0)
#    ht_average_arr = np.mean(ht_arr, axis=0)
#    
#    return h_average_arr, ht_average_arr

In [5]:
def filter_data(data_in, max_step_size, window, restriction=-1):
    """
    Only keeps data for tracks that are long enough for hurst exponent estimation at a given downsampling step size.
    Args: 
        data_in: pandas dataframe, original data
        max_step_size: int, maximum downsampling step size
        window: int, size of rolling window for hurst component estimation
    Returns:
        filtered_data: pandas dataframe
    """
    tracks_to_keep = data_in.TrackID.value_counts().loc[lambda x: (x//max_step_size) > (window+1)].reset_index()['index']
    if restriction<len(tracks_to_keep):
        filtered_data = data_in[data_in['TrackID'].isin(tracks_to_keep[:restriction])]
    else:
        filtered_data = data_in[data_in['TrackID'].isin(tracks_to_keep)]
    return filtered_data

In [6]:
def get_h_values(filtered_data, step_size, window, restriction):
    """
    Args:
        filtered_data: pandas dataframe
    Returns 1D array of (mean) hurst exponent values for a given step size.
    """
    track_id_values = np.unique(filtered_data['TrackID'])
    
    h = np.array([])
    for tid in track_id_values:
        track_data = filtered_data[filtered_data['TrackID']==tid]
        h_arr, ht_arr = dsample_est_hurst(track_data, step_size, window)  
        # h_av_arr, ht_av_arr = average_hurst(h_arr, ht_arr)  
        h = np.append(h, np.ravel(h_arr))
        
    return h

In [38]:
def get_h_dict(filtered_data, step_sizes, window, restriction):
    """
    """
    h_dict = {}
    
    for i, s in enumerate(step_sizes):
        h_arr = get_h_values(filtered_data, s, window, restriction)
        h_dict["{}".format(s)] = h_arr.tolist()
        
    return h_dict

def get_hist_h(h_dictionary, nbins):
    """
    """
    keys_list = list(h_dictionary)
    
    counts_all = np.empty((len(keys_list), nbins))
    bins_all = np.empty((len(keys_list), nbins+1))
    
    for i,key in enumerate(keys_list):
        counts, bins = np.histogram(h_dictionary[key], nbins, density=True)  # normalised so area under histogram is 1
        counts_all[i] = counts
        bins_all[i] = bins
    
    return counts_all, bins_all

def save_h_data(h_data, file_name, window, step_sizes, restriction):
    """
    """
    with open('h_dict_'+'w{}_'.format(window)+'s{}_'.format(max(step_sizes))+'r{}_'.format(restriction)+file_name, 'w') as file:
        file.write(json.dumps(h_data))
        
def get_dict_from_file(file_name, window, step_sizes, restriction,path='h_dict_data/'):
    """
    Reads in dictionary of H values.
    """
    try:
        with open(path+'h_dict_'+'w{}_'.format(window)+'s{}_'.format(max(step_sizes))+'r{}_'.format(restriction)+file_name, 'r') as file:
            h_dict = file.read()
    except FileNotFoundError:
        with open(path+'h_dict_'+'w{}_'.format(window)+'s{}_'.format(max(step_sizes))+'r{}_'.format(restriction)+file_name+'.txt', 'r') as file:
            h_dict = file.read()
        
    return json.loads(h_dict)

### Number of tracks available for given window size and step sizes 1, 2, ..., 10

In [39]:
def number_of_tracks(filenames, window, step_sizes=np.array([1,2,3,4,5,6,7,8,9,10]), pandas_df=True):
    """
    """
    ntracks_dict = {}
    
    for i, file in enumerate(filenames):
        data = pd.read_csv('haemocyte_tracking_data/'+file+'.csv')
        ntracks = np.empty((len(step_sizes)+1))
        ntracks[0] = len(np.unique(data['TrackID']))  # number of tracks in original data (independent of window, step size)
        for j, s in enumerate(step_sizes):
            filtered_data = filter_data(data, s, window)
            ntracks[j+1] = len(np.unique(filtered_data['TrackID']))  # number of tracks in filtered data
        ntracks_dict[file] = ntracks.tolist()
    
    if not pandas_df:
        return ntracks_dict
    
    else:
        column_labels = np.array(['original'])
        column_labels = np.append(column_labels, step_sizes.astype(str))
        df = pd.DataFrame.from_dict(ntracks_dict, orient='index', columns=column_labels)
        return df.apply(pd.to_numeric, downcast='integer')

In [34]:
window = 10

filenames = np.array(['Control_frame001-200',
                      'Control_frame200-400',
                      'Control_frame400-600',
                      'Control_frame600-800',
                      'Control_frame800-1000',
                      'Control_frame1000-1200',
                      'LanB1_frame001-200',
                      'LanB1_frame200-400',
                      'LanB1_frame400-600',
                      'LanB1_frame600-800',
                      'LanB1_frame800-1000',
                      'LanB1_frame1000-1200',
                      'defLanB1_300817_frame200-400',
                      'defLanB1_300817_frame400-600'])

print('for a window size of {}:'.format(window))
number_of_tracks(filenames, window)

for a window size of 10:


,original,1,2,3,4,5,6,7,8,9,10
Control_frame001-200,1806,1789,1208,765,517,348,246,171,135,99,77
Control_frame200-400,1468,1466,1149,819,621,448,332,252,192,146,116
Control_frame400-600,2172,2170,1815,1308,1000,728,563,433,364,292,248
Control_frame600-800,2117,2111,1833,1344,992,765,615,510,415,347,288
Control_frame800-1000,1725,1724,1558,1220,945,749,602,514,430,342,307
Control_frame1000-1200,1624,1623,1469,1217,944,740,596,477,395,332,282
LanB1_frame001-200,4793,2080,1124,665,416,280,210,144,108,83,72
LanB1_frame200-400,7239,2818,1549,994,671,469,350,278,223,174,140
LanB1_frame400-600,3091,1439,914,660,508,415,336,284,225,189,160
LanB1_frame600-800,5046,2247,1419,1050,793,643,546,461,393,336,297


### Main

In [10]:
#load neural network model
def load_nn_model(window_size,n_samples=100,n_epochs=100):

    try:
        model = tf.keras.models.load_model("model3dense_n{}.h5".format(window_size))
    except OSError:
        fbm_nn.__main__(n_samples,window_size,n_epochs)
        model = tf.keras.models.load_model("model3dense_n{}.h5".format(window_size))
    return model 

model = load_nn_model(window)


In [30]:
step_sizes = [1,2,3,4,5,6,7,8]
restriction = 30

#filenames = ['Control_frame200-400', 'LanB1_frame200-400']

# iterate over data files and save H dictionary 
for i, file in enumerate(filenames[4:]):
    print('opened file {}'.format(file))
    data = pd.read_csv('haemocyte_tracking_data/' + file + '.csv')
    filtered_data = filter_data(data, max(step_sizes), window, restriction)
    h_dict = get_h_dict(filtered_data, step_sizes, window, restriction)
    save_h_data(h_dict, file, window, step_sizes, restriction)

opened file Control_frame800-1000


NameError: name 'model' is not defined

## Scrap

In [54]:
#### OLD h down sampling + averaging 
"""
def get_hht_avg_dict(data_in, step_sizes):
    """
    """
    h_av_dict = {}
    #ht_av_dict = {}
    
    for i, s in enumerate(step_sizes):
        h_arr, ht_arr = dsample_est_hurst(data_in, s)
        h_av_arr, ht_av_arr = average_hurst(h_arr, ht_arr)
    
        h_av_dict["{}".format(s)] = h_av_arr.tolist()
     #   ht_av_dict["{}".format(s)] = ht_av_arr.tolist()
        
    return h_av_dict#, ht_av_dict
    

def save_hht_data(h_avg_data,ht_avg_data):
    """
    """
    with open('h_avg_file.txt', 'w') as h_avg_file:
        h_avg_file.write(json.dumps(h_avg_data))
        
    with open('ht_avg_file.txt', 'w') as ht_avg_file:
        ht_avg_file.write(json.dumps(ht_avg_data))
"""
#### 2 and 3D Plotting -- moved to haemocyte_plot
### frist block is really old 
        
""""    

%matplotlib notebook
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(projection="3d")

cmap = plt.cm.plasma

yticks = np.array(step_sizes)

for i, s in enumerate(step_sizes):
    ytick = yticks[i]
    normalised_counts = hist[0][i]
    bins = hist[1][i]
    xcenter = np.convolve(bins, np.ones(2), "valid")/2
    xwidth = np.diff(bins)
    ax.bar(left=xcenter, height=normalised_counts, width=xwidth, zs=ytick,
           zdir="y", color=cmap(i/len(yticks)), alpha=0.6, edgecolor="grey", linewidth=0.3)
    
ax.set_xlabel("H")
ax.set_ylabel("downsampling step size")
ax.set_zlabel("P(H)")

ax.set_yticks(yticks)
plt.show()


def plot_3d(h_dict,n_bins):
    
    %matplotlib notebook
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(projection="3d")
    cmap = plt.cm.plasma
    
    steps = [int(s) for s in list(h_dict)]
    yticks = np.array(steps)
    
    norm_counts, bins = get_hist_h(h_dict,n_bins)

    for i,_ in enumerate(steps):
        ytick = yticks[i]
        temp_counts = norm_counts[i]
        temp_bins = bins[i]
        xcenter = np.convolve(temp_bins, np.ones(2), "valid")/2
        xwidth = np.diff(temp_bins)
        ax.bar(left=xcenter, height=temp_counts, width=xwidth, zs=ytick,
               zdir="y", color=cmap(i/len(yticks)), alpha=0.6, edgecolor="grey", linewidth=0.3)
       

    ax.set_xlabel("H")
    ax.set_ylabel("downsampling step size")
    ax.set_zlabel("P(H)")

    ax.set_yticks(yticks)
    plt.show()
    
    return norm_counts,bins

def plot_hist_2D(data, step_sizes, nbins, window):
    """
    
    """
    track_id_values = np.unique(data['TrackID'])
    
    # plot histogram for each downsampling step size
    for s in step_sizes:
        # compute H values for each track and put them all in one array
        h = np.array([])
        for tid in track_id_values:
            track_data = data[data['TrackID']==tid]
            h_arr, ht_arr = dsample_est_hurst(track_data, s, window)  
            # h_av_arr, ht_av_arr = average_hurst(h_arr, ht_arr)  
            h = np.append(h, np.ravel(h_arr))
        plt.hist(h, nbins, density=True, label='step size = {}'.format(s))  
        # density=True for normalisation so that area under histogram integrates to 1
    plt.legend()
    plt.xlabel('H')
    plt.ylabel('p(H)')
    plt.show()
    
step_sizes = [1,2,3,4,5,6,7,8]
h_dict_temp = get_dict_from_file('LanB1_frame1000-1200',10,step_sizes,30)
norm_counts,bins = plot_3d(h_dict_temp,25)
        
"""
#### hurst vs time plot -- Not sure if this has been moved anywhere else???
"""
def plot_downsampled_hurst(h_arr, ht_arr):
    """
    """
    h_av_arr, ht_av_arr = average_hurst(h_arr, ht_arr)
    
    for i in np.arange(len(h_arr)):
        plt.plot(ht_arr[i], h_arr[i], alpha=0.5)
    plt.plot(ht_av_arr, h_av_arr, label='mean H', color='k')
    plt.axhline(0.5, c='r')
    plt.xlabel('t')
    plt.ylabel('H')
    plt.title('Downsampled data, step size {}'.format(len(h_arr)))
    plt.legend()
    plt.show()

"""

IndentationError: unexpected indent (<ipython-input-54-d39c11e44a24>, line 5)

In [56]:
####GMM code old

"""
# first of all, let's confirm the optimal number of components
def optimum_components(data,n_max,plot=0):
    bics = []
    min_bic = 0
    counter=1
    for i in range (n_max-1): # test the AIC/BIC metric between 1 and 10 components
        gmm = GMM(n_components = counter, max_iter=1000, random_state=0, covariance_type = 'full')
        labels = gmm.fit(data).predict(data)
        bic = gmm.bic(data)
        bics.append(bic)
        if bic < min_bic or min_bic == 0:
            min_bic = bic
            opt_bic = counter
        counter +=1
    
    if plot==1:    
        fig,ax = plt.subplots(figsize=(10, 4))
        # Plot 1
        plt.plot(np.arange(1,n_max), bics, 'o-', lw=3, c='black', label='BIC')
        plt.legend(frameon=False, fontsize=15)
        plt.xlabel('Number of components', fontsize=12)
        plt.ylabel('Bayesian Information criterion', fontsize=12)
        plt.xticks(np.arange(0,n_max, 2))
        plt.title('Opt. components = '+str(opt_bic), fontsize=20)
        plt.show()

        
    return opt_bic,bics

data_temp = np.concatenate((np.random.normal(5, 1, 1000), np.random.normal(10, 2, 1000),np.random.normal(15, 3, 1000)))
data_temp = data_temp.reshape(-1, 1)   #hist[1][1].reshape(-1,1)
n_optimum,bic_vals = optimum_components(data_temp,10,plot=1)

fig,ax = plt.subplots()
plt.hist(data_temp,100)
plt.show()

# create GMM model object
gmm = GMM(n_components = n_optimum, max_iter=1000, random_state=10, covariance_type = 'full')

# find useful parameters
mean = gmm.fit(data_temp).means_  
covs  = gmm.fit(data_temp).covariances_
weights = gmm.fit(data_temp).weights_

x_axis = np.arange(-50, 50, 0.1)

# create necessary things to plot
g1 = norm.pdf(x_axis, float(mean[0][0]), np.sqrt(float(covs[0][0][0])))*weights[0] # 1st gaussian
g2 = norm.pdf(x_axis, float(mean[1][0]), np.sqrt(float(covs[0][0][0])))*weights[1] # 1st gaussian
g3 = norm.pdf(x_axis, float(mean[2][0]), np.sqrt(float(covs[0][0][0])))*weights[2] # 1st gaussian

fig,ax = plt.subplots(figsize=(9,8))
# Plot 2
plt.hist(data_temp, density=True, color='black',bins=100)
plt.plot(x_axis, g1, c='r')
plt.plot(x_axis, g2, c='g')
plt.plot(x_axis, g3, c='b')
plt.xlabel(r"X", fontsize=20)
plt.ylabel(r"Density", fontsize=20)

plt.show()

"""

'\n# first of all, let\'s confirm the optimal number of components\ndef optimum_components(data,n_max,plot=0):\n    bics = []\n    min_bic = 0\n    counter=1\n    for i in range (n_max-1): # test the AIC/BIC metric between 1 and 10 components\n        gmm = GMM(n_components = counter, max_iter=1000, random_state=0, covariance_type = \'full\')\n        labels = gmm.fit(data).predict(data)\n        bic = gmm.bic(data)\n        bics.append(bic)\n        if bic < min_bic or min_bic == 0:\n            min_bic = bic\n            opt_bic = counter\n        counter +=1\n    \n    if plot==1:    \n        fig,ax = plt.subplots(figsize=(10, 4))\n        # Plot 1\n        plt.plot(np.arange(1,n_max), bics, \'o-\', lw=3, c=\'black\', label=\'BIC\')\n        plt.legend(frameon=False, fontsize=15)\n        plt.xlabel(\'Number of components\', fontsize=12)\n        plt.ylabel(\'Bayesian Information criterion\', fontsize=12)\n        plt.xticks(np.arange(0,n_max, 2))\n        plt.title(\'Opt. componen